$\textbf{GOAL}$ : We want to get a correct CD algorithm

In [1]:
using LinearAlgebra, Printf, Statistics, Random

# Coordinate Descent (CD)


Let $f:\mathbb R^{n}\to\mathbb R$ $C^1$, convex and with gradient of f $L_{max}$-Lispschitz. Then a penalty function separable $g$ such as we want minimize $F:=f+\lambda g$, for a hyperparameter $\lambda$, 

Typical choices of separable penalty $\Omega_i(t)=|t|$ ($\ell\_1$‐norm), $\Omega_i=\iota_{[a_i,b_i]}$ (indicator of a coordinate box)

For each \$i\in{1,\dots,n}\$ define the smallest \$L\_i>0\$ such that, $\forall x\in\mathbb R^{n},t\in\mathbb R$, 
$
\bigl|[\partial f(x+te_i)]_i-[\partial f(x)]_i\bigr|
\leq L_i|t|
$
and we set  $L_{\max}:=\max_{1\le i\le n}L_i$


**Property** (Pourquoi celle là ? jsp mais je la trouve intéressante donc je la note) [**Strong** convexity]
If there exists, $\forall x,y$,  $\sigma>0$ with
$$ f(y)\ge f(x)+\partial f(x)^{\top}(y-x)+\tfrac{\sigma}{2}\|y-x\|_2^{2}$$
then $f$ is $ \sigma $-strongly convex.

*We prefer a large $\sigma$ to converge faster (strong curvature) than low $sigma$. Moreover, we are only convex if $\sigma=0$.*


The main idea of the method is to optimize one dimension at the time. So, we fix $i \in \llbracket 1,n\rrbracket$ and we optimise from $x\in \mathbb{R}^n$ to $y:=x+(\chi-x_i)e_i$ where $\chi \in \mathbb{R}$. We have to find $\chi$.

By Lipschitzity of the gradient of L using Taylor expansion,   
$$
f\bigl(y)\le\ f(x)+ [\nabla f(x)]_i(\chi-x_i) + \frac{L_i}{2}(\chi-x_i)^2
$$
 
So,
$[F(y)-F(x)]_i \leq  [\partial f(x)]_i(\chi-x_i)+\frac{L_i}{2}(\chi-x_i)^2 +\lambda\bigl[\Omega_i(\chi)-\Omega_i(x_i)\bigr] $.
Then, we want the smallest upper bound, that's to say for $\alpha\le 1/L_i$,
$$\chi^* := \argmin_\chi [\partial f(x)]_i(\chi-x_i)+\frac{1}{2\alpha}(\chi-x_i)^2+\lambda\Omega_i(\chi) $$
So we have for this $\chi^*$ (optimal) descending iteration $F(y)-F(x) \leq 0 \Longleftrightarrow F(x+(\chi^*-x_i)e_i) \leq F(x)$

We can remark that, $$\chi^* = \operatorname{prox}_{\alpha,\lambda\Omega_i}\!\bigl(x_i-\alpha[\nabla f(x)]_i\bigr)$$

In the following we use the cyclic algorithm 2 of
> Coordinate Descent Algorithms, Stephen J. Wright *(https://arxiv.org/pdf/1502.04759)*

# LASSO

To resolve LASSO, we need to find these $L_i$ and the explicit form of the proximal. 

Let $X \in \mathbb{R}^{n \times p}$ and $y \in \mathbb{R}^{n}$

$$
\min_{b \in \mathbb{R}^{p}}
\frac12 \|y - X b\|_{2}^{2}
+ \lambda \|b\|_{1}
\tag{LASSO}
$$

So in this first case, we have a optimization problem under the form $f+g$ with $f=MSE$ and $g = \ell_1$-norm 

We have for each coordinate $i$, $\partial f(x)_i=X_i^{\top}(Xb-y)$ *($X_i$ represent the i-th column of X)*

$$[\partial f(x+te_i)]_i-[\partial f(x)]_i=X_i^{\top}\!\bigl(X(b+te_i)-y\bigr)-X_i^{\top}(Xb-y)=tX_i^{\top}X_i=t\|X_i\|_2^{2}
$$
So $\forall x,t$ and setting $L_i:=\|X_i\|_2^{2}$ 

$$|[\partial f(x+te_i)]_i-[\partial f(x)]_i|\le L_i|t|$$

## 


$\textbf{Proposition} $
Let $x\in\mathbb R^{n}$, an index i and a step $0<\alpha\le \frac{1}{L_i}$ with $u:=x_i-\alpha[\partial f(x)]_i$  
So, $$\chi^* =\operatorname{sign}(u)\max\{|u|-\alpha\lambda,0\}$$

*Proof.*  
$$
   [\partial f(x)]_i(\chi-x_i) + \frac{1}{2\alpha}(\chi-x_i)^2= 
   \frac{1}{2\alpha}\bigl(\chi - (x_i-\alpha[\partial f(x)]_i)\bigr)^2- \frac{\alpha}{2}[\partial f(x)]_i^{2}
   = \frac{1}{2\alpha}(\chi-u)^2 - \frac{\alpha}{2}[\partial f(x)]_i^{2}
$$
 
The minimisation reduces to (last term is constant according to $\chi$)  
$$
     \chi^\star = \argmin_{\chi}\Bigl\{ \tfrac12(\chi-u)^2 + \alpha\lambda|\chi| \Bigr\} = \operatorname{prox}_{\alpha,\lambda|\cdot|}(u)
$$

And we know develop the expression of this proximal : See ISTA.ipynb (in RAPHAEL) 


# General function (f, g, L, prox given)

Can be improve because we could pass an array of function for the gradient of f for instance instead of computing the gradient of f on all coordinate and take only the i-th coordinate.

In the project we don't use massively, only to assure ISTA algorithm converges so we don't need performance.

In [14]:
function cd_L(x0, f, g, ∇f, L, prox;
        max_iter = 100_000, tol = 1e-9,print_freq = 1000, verbose = true
    )
    x = copy(x0)
    n = length(x)
    Li = (L isa AbstractVector) ? L : fill(L, n) # is an array. Each entry for each coordinate
    cost_prev = f(x) +g(x)
    epoch = 0

    for k in 1:max_iter
        i = (mod(k-1, n) + 1) # cyclic

        grad_i = ∇f(x)[i] 
        step = 1 / Li[i]
        z_i = prox(x[i] - step * grad_i, step)
        x[i] = z_i 
        if i == n # finished a full sweep (one epoch)
            epoch += 1

            cost_current = f(x) + g(x)
            diff         = abs(cost_current - cost_prev)

            if verbose && (epoch == 1 || epoch % print_freq == 0)
                @printf("[CD]  epoch %6d  cost = %.3e   diff = %.3e\n",epoch, cost_current, diff)
            end

            if diff < tol
                if verbose
                    @printf("[CD]  END   epoch %6d  cost = %.3e   diff = %.3e\n",epoch, cost_current, diff)
                end
                break
            end
            cost_prev = cost_current
        end
    end
    return x
end

cd_L (generic function with 1 method)

## Test

In [15]:
n, p = 100, 50
Random.seed!(42)
X = randn(n, p)
sigma = 0.1
y = X * randn(p) + sigma * randn(n)
λ = 0.1

0.1

### LASSO

In [16]:
f(b) = 0.5*norm(y-X*b,2)^2
∇f(b) = -X'*(y - X*b)
g(x) = λ*sum(abs, x) # L1 norm
prox_L(x, step) = sign.(x) .* max.(abs.(x) .- λ*step, zero(x))
l_L = [norm(X[:, j],2)^2 for j in 1:p]
Lmax = maximum([norm(X[:, j])^2 for j in 1:p])


beta0 = zeros(p)# initial conditions
beta_L_l = cd_L(beta0, f, g, ∇f, l_L, prox_L, tol=1e-9)
beta_L_max = cd_L(beta0, f, g, ∇f, Lmax, prox_L, tol=1e-9)

[CD]  epoch      1  cost = 3.806e+02   diff = 2.142e+03
[CD]  END   epoch     46  cost = 4.280e+00   diff = 8.367e-10
[CD]  epoch      1  cost = 3.480e+02   diff = 2.175e+03
[CD]  END   epoch     81  cost = 4.280e+00   diff = 9.842e-10


50-element Vector{Float64}:
 -0.22167751066748717
 -0.8107640338613908
  1.2197351546784327
  2.1296541867440673
  0.8582180832818391
 -0.31706472256486956
  0.6494017717295051
 -0.2791655374119629
  1.1051331043116277
 -0.1107449431558722
  ⋮
  0.26028211088965736
  2.000041131330912
 -0.27439979683330834
 -1.1161814851235266
 -1.231318360750697
  0.11211728011527106
 -0.9711042311174327
 -0.32961665412223606
 -0.15896085918977879

It's good ! We have same results than ISTA and using the adapated L constants instead of Lmax is accelarating well convergence of the algorithm !

## Comparisons

In [5]:
include("../../comp_translate/SaveLoadTxt.jl") # Assuming you have a SaveLoadTxt.jl file with the necessary functions
using .SaveLoadTxt

### Setup

In [6]:
n, p = 100, 50
sigma = 0.1
X, y = load_txt("../../comp_translate/data/ISTA.txt")
λ = 0.1
;

### Results

In [7]:
f(b) = 0.5*norm(y-X*b,2)^2
∇f(b) = -X'*(y - X*b)
g(x) = λ*sum(abs, x) # L1 norm
prox_L(x, step) = sign.(x) .* max.(abs.(x) .- λ*step, zero(x))
l_L = [norm(X[:, j])^2 for j in 1:p]
Lmax = maximum([norm(X[:, j])^2 for j in 1:p])


beta0 = zeros(p)# initial conditions
beta_L_l = cd_L(beta0, f, g, ∇f, l_L, prox_L, tol=1e-9)
beta_L_max = cd_L(beta0, f, g, ∇f, Lmax, prox_L, tol=1e-9)

[CD]  epoch      1  cost = 3.806e+02   diff = 2.142e+03
[CD]  END   epoch     46  cost = 4.280e+00   diff = 8.367e-10
[CD]  epoch      1  cost = 3.480e+02   diff = 2.175e+03
[CD]  END   epoch     81  cost = 4.280e+00   diff = 9.842e-10


50-element Vector{Float64}:
 -0.22167751066748717
 -0.8107640338613908
  1.2197351546784327
  2.1296541867440673
  0.8582180832818391
 -0.31706472256486956
  0.6494017717295051
 -0.2791655374119629
  1.1051331043116277
 -0.1107449431558722
  ⋮
  0.26028211088965736
  2.000041131330912
 -0.27439979683330834
 -1.1161814851235266
 -1.231318360750697
  0.11211728011527106
 -0.9711042311174327
 -0.32961665412223606
 -0.15896085918977879

# General function BackTrack *(f, g, $L_0$, prox given)*